<a href="https://colab.research.google.com/github/moraeseg/GitTutorial/blob/main/clsLerArquivoSequencial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import csv
from enum import Enum
from typing import Optional, Dict, List, Any, Tuple, Callable
import re

class AcaoArquivo(Enum):
    LEITURA = 1
    CONVERSAO_CSV = 2
    EXTRACT_LINHAS = 3


class ArquivoROCO:
    def __init__(self,
                 nome_arquivo_origem = None,
                 path_arquivo_origem = None,
                 dicionario_arquivo = None):
        self.nome_arquivo_origem = nome_arquivo_origem
        self.path_arquivo_origem = path_arquivo_origem
        # O dicionario_arquivo, que contém os nomes das colunas, é um atributo da instância
        # e está acessível em todos os métodos usando self.dicionario_arquivo
        self.dicionario_arquivo = dicionario_arquivo
        self._dados_lidos = None # Atributo para armazenar os dados lidos

    def _apply_filter(self, registro: Dict[str, str], filtro_str: str) -> bool:
        """
        Aplica um filtro baseado em string (SQL WHERE-like) a um registro.
        Suporta operadores ==, !=, >, <, >=, <= e LIKE.
        Formato esperado: "nome_coluna operador 'valor'".
        LIKE suporta '%' como curinga.
        Retorna True se o registro deve ser incluído, False se deve ser excluído.
        """
        if not filtro_str:
            return True  # Sem filtro, incluir todos os registros

        # Regex para capturar nome da coluna, operador e valor
        match = re.match(r"^\s*(\w+)\s*([=!<>]+|LIKE)\s*['\"]?(.*?)['\"]?\s*$", filtro_str)
        if not match:
            print(f"Aviso: Formato de filtro inválido: {filtro_str}")
            return True # Incluir registro se o filtro for inválido para evitar perda de dados

        coluna, operador, valor_str = match.groups()

        if coluna not in registro:
            print(f"Aviso: Coluna '{coluna}' não encontrada no registro para aplicar o filtro.")
            return True # Incluir registro se a coluna não existir

        valor_registro = registro.get(coluna, "").strip() # Obter valor do registro e remover espaços

        # Tenta converter para numérico para comparações >, <, etc.
        try:
            valor_registro_num = float(valor_registro)
            valor_filtro_num = float(valor_str)
            is_numeric_comparison = True
        except ValueError:
            is_numeric_comparison = False

        if operador == "==":
            return valor_registro == valor_str
        elif operador == "!=":
            return valor_registro != valor_str
        elif operador == "LIKE":
            # Converter padrão LIKE para regex
            pattern = valor_str.replace('%', '.*')
            return re.search(pattern, valor_registro) is not None
        elif is_numeric_comparison:
            if operador == ">":
                return valor_registro_num > valor_filtro_num
            elif operador == "<":
                return valor_registro_num < valor_filtro_num
            elif operador == ">=":
                return valor_registro_num >= valor_filtro_num
            elif operador == "<=":
                return valor_registro_num <= valor_filtro_num
        else:
             print(f"Aviso: Operador '{operador}' não suportado ou tipo de dado incompatível para a comparação no filtro: {filtro_str}")
             return True # Incluir registro se o operador não for suportado ou tipo de dado incompatível

        return True # Incluir por padrão se nenhuma condição for atendida (pode ser ajustado)


    def run_acao(self,
                 acao_no_arquivo: AcaoArquivo,
                 filtro: Optional[str] = None) -> Optional[List[Dict[str, str]]]:
        """
        Executa a ação especificada no arquivo, lendo-o posicionalmente
        de acordo com o dicionario_arquivo e retornando os dados.
        Opcionalmente aplica um filtro baseado em string para incluir/excluir registros.
        O dicionario_arquivo (com nomes de colunas) está acessível via self.dicionario_arquivo.
        """
        if (self.dicionario_arquivo and
            self.path_arquivo_origem and
            self.nome_arquivo_origem):

            full_path = self.path_arquivo_origem + self.nome_arquivo_origem
            dados_lidos = []
            try:
                with open(full_path, 'r') as f:
                    for linha in f:
                        linha = linha.strip()
                        registro = {}
                        # Lendo todas as colunas primeiro antes de aplicar o filtro
                        for nome_coluna, (inicio, fim) in self.dicionario_arquivo.items():
                             # Note: O filtro agora é para registros, não colunas individuais na leitura posicional inicial
                            registro[nome_coluna] = linha[inicio:fim]

                        # Aplica o filtro ao registro lido
                        if filtro is None or self._apply_filter(registro, filtro):
                           dados_lidos.append(registro) # Adiciona o registro se não houver filtro ou se o filtro for True

                self._dados_lidos = dados_lidos # Armazena os dados lidos na variável de instância

                if acao_no_arquivo == AcaoArquivo.LEITURA:
                    return self._dados_lidos
                elif acao_no_arquivo == AcaoArquivo.CONVERSAO_CSV:
                    self._converter_para_csv_interno() # Chama o método de conversão interno
                    return None # Retorna None ou uma mensagem de sucesso após a conversão
                elif acao_no_arquivo == AcaoArquivo.EXTRACT_LINHAS:
                    print("Ação de extração de linhas solicitada. "
                          "Funcionalidade ainda não implementada.")
                    return None # Funcionalidade a ser implementada
                else:
                    print("Ação não especificada.")
                    return None


            except FileNotFoundError:
                print(f"Erro: Arquivo não encontrado em {full_path}")
                return None
            except Exception as e:
                print(f"Ocorreu um erro ao ler o arquivo: {e}")
                return None
        else:
             print("Dicionário, caminho do arquivo ou nome do arquivo faltando.")
             return None

    def _converter_para_csv_interno(self,
                                    nome_arquivo_destino="saida.csv",
                                    path_arquivo_destino="/content/"):
        """
        Converte os dados lidos (armazenados em self._dados_lidos) para o formato CSV
        e salva em um arquivo. Este é um método interno chamado por run_acao.
        O dicionario_arquivo (com nomes de colunas) está acessível via self.dicionario_arquivo.
        Note: O filtro aplicado na leitura (run_acao) já determinou quais registros estão em _dados_lidos.
              Este método apenas escreve os dados já filtrados, se houver.
        """
        if not self._dados_lidos:
            print("Nenhum dado para converter (dados lidos não disponíveis ou nenhum registro passou no filtro).")
            return

        full_path_destino = path_arquivo_destino + nome_arquivo_destino

        try:
            with open(full_path_destino, 'w', newline='') as csvfile:
                # Acessando nomes das colunas através de self.dicionario_arquivo
                # Note: Este método não aplica filtro de colunas, apenas de registros (feito em run_acao)
                fieldnames = list(self.dicionario_arquivo.keys())
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                writer.writeheader()
                for registro in self._dados_lidos:
                    writer.writerow(registro)
            print(f"Dados convertidos e salvos em {full_path_destino}")
        except Exception as e:
            print(f"Ocorreu um erro ao salvar o arquivo CSV: {e}")


# O método converter_para_csv original pode ser removido ou adaptado se ainda for necessário
# def converter_para_csv(self, dados, ...):
#    pass


# Exemplo de uso:
# Para testar a leitura e conversão para CSV, você precisará criar um arquivo de exemplo.
# Crie um arquivo chamado 'teste.txt' no diretório '/content/sample_data/'
# com conteúdo posicional, por exemplo:
# 12345abcde
# 67890fghij
# onde 'coluna1' é de 0 a 5 e 'coluna2' é de 5 a 10.

# Exemplo de criação de um arquivo temporário para teste (em um ambiente real, o arquivo já existiria)
import os
if not os.path.exists("/content/sample_data/"):
   os.makedirs("/content/sample_data/")
with open("/content/sample_data/teste.txt", "w") as f:
   f.write("12345abcde\n")
   f.write("67890fghij\n")
   f.write("outra\n") # Adicionando uma linha que pode não ter todas as colunas

# Exemplo de uso com Enum para leitura sem filtro
arquivo_roco_leitura = ArquivoROCO(nome_arquivo_origem="teste.txt",
                                   path_arquivo_origem="/content/sample_data/",
                                   dicionario_arquivo={"coluna1": (0, 5), "coluna2": (5, 10), "coluna3": (10, 15)})
dados_lidos = arquivo_roco_leitura.run_acao(AcaoArquivo.LEITURA)
if dados_lidos:
    print("\nDados lidos sem filtro de registro:")
    for registro in dados_lidos:
        print(registro)

# Exemplo de uso com Enum para leitura com filtro (coluna1 == '12345')
arquivo_roco_leitura_filtro = ArquivoROCO(nome_arquivo_origem="teste.txt",
                                          path_arquivo_origem="/content/sample_data/",
                                          dicionario_arquivo={"coluna1": (0, 5), "coluna2": (5, 10), "coluna3": (10, 15)})

dados_lidos_filtro = arquivo_roco_leitura_filtro.run_acao(AcaoArquivo.LEITURA, filtro="coluna1 == '12345'")
if dados_lidos_filtro:
    print("\nDados lidos com filtro (coluna1 == '12345'):")
    for registro in dados_lidos_filtro:
        print(registro)
else:
    print("\nNenhum dado lido com filtro (coluna1 == '12345')")


# Exemplo de uso com Enum para leitura com filtro (coluna2 LIKE '%ghi%')
arquivo_roco_leitura_filtro_like = ArquivoROCO(nome_arquivo_origem="teste.txt",
                                               path_arquivo_origem="/content/sample_data/",
                                               dicionario_arquivo={"coluna1": (0, 5), "coluna2": (5, 10), "coluna3": (10, 15)})

dados_lidos_filtro_like = arquivo_roco_leitura_filtro_like.run_acao(AcaoArquivo.LEITURA, filtro="coluna2 LIKE '%ghi%'")
if dados_lidos_filtro_like:
    print("\nDados lidos com filtro (coluna2 LIKE '%ghi%'):")
    for registro in dados_lidos_filtro_like:
        print(registro)
else:
     print("\nNenhum dado lido com filtro (coluna2 LIKE '%ghi%')")


# Exemplo de uso com Enum para conversão para CSV sem filtro de registro
arquivo_roco_conversao = ArquivoROCO(nome_arquivo_origem="teste.txt",
                                     path_arquivo_origem="/content/sample_data/",
                                     dicionario_arquivo={"coluna1": (0, 5), "coluna2": (5, 10), "coluna3": (10, 15)})
arquivo_roco_conversao.run_acao(AcaoArquivo.CONVERSAO_CSV, nome_arquivo_destino="saida_conversao_sem_filtro_registro.csv")

# Exemplo de uso com Enum para conversão para CSV com filtro de registro (coluna1 > '60000')
arquivo_roco_conversao_filtro = ArquivoROCO(nome_arquivo_origem="teste.txt",
                                            path_arquivo_origem="/content/sample_data/",
                                            dicionario_arquivo={"coluna1": (0, 5), "coluna2": (5, 10), "coluna3": (10, 15)})
arquivo_roco_conversao_filtro.run_acao(AcaoArquivo.CONVERSAO_CSV,
                                       nome_arquivo_destino="saida_conversao_com_filtro_registro.csv",
                                       filtro="coluna1 > '60000'")

# Exemplo de uso com Enum para extração de linhas (funcionalidade não implementada)
arquivo_roco_extract = ArquivoROCO()
arquivo_roco_extract.run_acao(AcaoArquivo.EXTRACT_LINHAS)

NameError: name 'self' is not defined

**Reasoning**:
Import the necessary libraries for creating a Flask application and handling requests, and import the previously defined `ArquivoROCO` class and `AcaoArquivo` enum.



**Reasoning**:
Include the code to run the Flask development server.



## Refinar a interface e o código

### Subtask:
Melhorar a experiência do usuário na interface e adicionar tratamento de erros tanto no frontend quanto no backend.


**Reasoning**:
Update the HTML with visual feedback and enhanced result display, and update the Python backend to include more detailed error handling and return more specific error messages.



**Reasoning**:
The previous attempt to remove the existing URL rule using `app.url_map.pop_by_endpoint` failed because the `Map` object does not have this method. In a notebook environment, redefining the function and then adding the rule again might still cause issues depending on how the environment manages the Flask app instance. A more reliable approach in this context is to define the updated function and simply use `app.add_url_rule`, trusting that the notebook environment handles the re-registration of the route correctly, or to restart the kernel if persistent issues occur. Given the error, the explicit removal is not the correct way. I will remove the explicit removal attempt and just define the updated function and add the rule.



## Summary:

### Data Analysis Key Findings

*   The HTML frontend was successfully created with input fields for file name, file path, action, and a textarea for the positional dictionary, along with a submit button and a div for results.
*   JavaScript was successfully added to the HTML to capture form data, parse the dictionary JSON (with basic error handling), and send the data to a backend endpoint using `fetch`.
*   The JavaScript was updated to target the correct backend endpoint `/run_arquivo_roco`.
*   The HTML was further refined to include a loading indicator and improved display of results and error messages from the backend.
*   A Python Flask backend endpoint `/run_arquivo_roco` was defined to receive POST requests with JSON data.
*   The backend code included logic to instantiate the `ArquivoROCO` class and call the `run_acao` method.
*   Attempts were made to enhance the Flask backend with more specific validation and error handling (e.g., for missing fields, invalid data types, `FileNotFoundError`, `IndexError`).
*   However, the attempts to update and redefine the Flask route in the execution environment consistently failed due to an `AssertionError` related to overwriting an existing endpoint function, preventing the enhanced backend logic from being fully applied and verified within the provided steps.

### Insights or Next Steps

*   To fully implement the enhanced backend error handling, the Flask application needs to be run in an environment where route redefinition is handled correctly, or the entire application needs to be defined and run in a single execution context to avoid the redefinition issue.
*   Further frontend validation could be added (e.g., ensuring file path format, checking if the dictionary is required based on the selected action before sending the request).
